In [ ]:
from pathlib import Path

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from helpers.base.loaders import prepare_data_for_pytorch
from helpers.base.models import FrequencyAwareNetwork
from helpers.base.trainers import train_model
from helpers.evals import evaluate_model
from helpers.features import process_dataset

### Config


In [2]:
ANALYSIS = False
VERBOSE = True

DATASET_FILE_PATH = "dataset.csv"

GRAPH_FOLDER = "graphs"
MODELS = "models"
PREDICTIONS = "predictions"
SUBFOLDER = "baseline"

### Data


In [3]:
df = pd.read_csv(DATASET_FILE_PATH)

(
    X_train,
    Y_train,
    X_test,
    Y_test,
    voltage_scaler,
    freq_scaler,
) = process_dataset(df, test_size=0.2, random_state=42)

### Training


In [4]:
best_hparams = {
    "hidden_sizes": [384, 768, 1536],
    "dropout_rate": 0.1,
    "learning_rate": 0.002,
    "batch_size": 2048,
    "epochs": 300,
    "patience": 40,
    "activation": "gelu",
    "lr_scheduler_type": "reduce_on_plateau",
}

model_dir = Path(MODELS) / SUBFOLDER
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
freq_idx = [X_train.columns.get_loc("freq")]
other_idx = [i for i in range(X_train.shape[1]) if i not in freq_idx]

results = {}

In [5]:
# Train one model for each label pair

s_parameter_groups = [
    ("S_deemb(1,1)_real", "S_deemb(1,1)_imag"),
    ("S_deemb(1,2)_real", "S_deemb(1,2)_imag"),
    ("S_deemb(2,1)_real", "S_deemb(2,1)_imag"),
    ("S_deemb(2,2)_real", "S_deemb(2,2)_imag"),
]

for real_label, imag_label in s_parameter_groups:
    label_pair = [real_label, imag_label]
    print(f"Training model for {real_label} + {imag_label}...")
    y_train_pair = Y_train[label_pair]
    y_test_pair = Y_test[label_pair]

    X_train_tensor, Y_train_tensor, X_test_tensor, Y_test_tensor, loader, _ = (
        prepare_data_for_pytorch(
            X_train,
            y_train_pair,
            X_test,
            y_test_pair,
            batch_size=best_hparams["batch_size"],
            scale_y=True,
        )
    )

    model = FrequencyAwareNetwork(
        freq_idx,
        other_idx,
        best_hparams["hidden_sizes"],
        best_hparams["dropout_rate"],
        best_hparams["activation"],
    )
    optimizer = optim.Adam(model.parameters(), lr=best_hparams["learning_rate"])
    criterion = nn.MSELoss()

    # save_path = f"{model_dir}/{real_label}_fan_model.pth"
    trained_model = train_model(
        model,
        loader,
        X_test_tensor,
        Y_test_tensor,
        criterion,
        optimizer,
        device,
        epochs=best_hparams["epochs"],
        patience=best_hparams["patience"],
        scheduler_str=best_hparams["lr_scheduler_type"],
    )
    metrics = evaluate_model(
        trained_model, X_test_tensor.to(device), Y_test_tensor.to(device)
    )
    results[f"{real_label}+{imag_label}"] = metrics

Training model for S_deemb(1,1)_real + S_deemb(1,1)_imag...


Training Epochs:  16%|█▋        | 49/300 [00:13<01:10,  3.54it/s, Epoch=49, Val Loss=548414.625000, Best=86191.187500, LR=2e-9]                          


Early stopping triggered.
Training model for S_deemb(1,2)_real + S_deemb(1,2)_imag...


Training Epochs:  18%|█▊        | 55/300 [00:15<01:06,  3.66it/s, Epoch=55, Val Loss=1.008730, Best=1.008719, LR=2e-9]                                 


Early stopping triggered.
Training model for S_deemb(2,1)_real + S_deemb(2,1)_imag...


Training Epochs:  15%|█▍        | 44/300 [00:11<01:08,  3.72it/s, Epoch=44, Val Loss=0.886364, Best=0.886364, LR=2e-9]                                   


Early stopping triggered.
Training model for S_deemb(2,2)_real + S_deemb(2,2)_imag...


Training Epochs:  25%|██▌       | 75/300 [00:20<01:01,  3.64it/s, Epoch=75, Val Loss=259623.890625, Best=44974.527344, LR=2e-9]                          

Early stopping triggered.


### Results


In [6]:
for label_pair, metrics in results.items():
    print(f"Performance for {label_pair}:")
    print(f"  R²:   {metrics['R2'][0]:.4f}, {metrics['R2'][1]:.4f}")
    print(f"  MAE:  {metrics['MAE'][0]:.4f}, {metrics['MAE'][1]:.4f}")
    print(f"  RMSE: {metrics['RMSE'][0]:.4f}, {metrics['RMSE'][1]:.4f}")


Performance for S_deemb(1,1)_real+S_deemb(1,1)_imag:
  R²:   -779000.4375, -362534.6875
  MAE:  736.9699, 492.5932
  RMSE: 870.5200, 582.2600
Performance for S_deemb(1,2)_real+S_deemb(1,2)_imag:
  R²:   -0.0037, -0.0047
  MAE:  0.7642, 0.8873
  RMSE: 0.9826, 1.0257
Performance for S_deemb(2,1)_real+S_deemb(2,1)_imag:
  R²:   -0.0014, -0.0003
  MAE:  0.5295, 0.7370
  RMSE: 0.9060, 0.9756
Performance for S_deemb(2,2)_real+S_deemb(2,2)_imag:
  R²:   -483108.7812, -38308.0781
  MAE:  589.2103, 156.4833
  RMSE: 696.5065, 184.6271
